## Model Test(Logistic regression, random forest, LightGBM)

In [1]:
# import the relevant computational modules

# data manipulation
import pandas as pd #data processing
import numpy as np #linear algebra

# Models Packages
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Gradient Boosting
import lightgbm as lgb
from sklearn.cross_validation import KFold

# grid search
from sklearn.model_selection import GridSearchCV

/home/data/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# import data

transaction_training = pd.read_csv('../raw_data/transactions_train.csv')
payment_training = pd.read_csv('../raw_data/paiements_train.csv')
billing_training = pd.read_csv('../raw_data/facturation_train.csv')
performance_training = pd.read_csv('../raw_data/performance_train.csv')


transaction_test = pd.read_csv('../raw_data/transactions_test.csv')
payment_test = pd.read_csv('../raw_data/paiements_test.csv')
billing_test = pd.read_csv('../raw_data/facturation_test.csv')
performance_test = pd.read_csv('../raw_data/performance_test.csv')

In [3]:
# Create basic scikit-learn wrapper model class
class SklearnWrapper:
    def __init__(self, clf, seed=0, params=None, seed_bool=True):
        if (seed_bool == True):
            params['random_state'] = seed
        self.clf = clf(**params)
    
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
        
    def predict(self, x):
        return self.clf.predict(x)

In [4]:
# create basic xgboost wrapper model class
class XgbWrapper:
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

In [5]:
# create basic lightGBM wrapper model class
class LightGbmWrapper:
    def __init(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 1550)
        self.verbose_eval = params.pop('verbose_eval', 100)
        
    def train(self, x_train, y_train):
        lgtrain = lgb.Dataset(x_train, y_train)
        self.lgbm = lgb.train(self.param, lgtrain, num_boost_round=self.nrounds, verbose_eval=self.verbose_eval)
    
    def predict(self, x):
        return self.lgbm.predict(lgb.Dataset(x))

In [6]:
# create out-of-fold predictions 
# make good use of k-fold CV's result 
# serving for the staking alogrithm 
# create a new column generated from model's score

def get_oof(clf, x_train, y, x_test):
    '''
    clf: the classifer, which can be logistic regression, SVM regression, Bayes classifier, etc.
    x_train: the training x in training dataset
    y: the training y in training dataset
    x_test: the testing x in training dataset 
    '''
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    for i, (train_index, test_index) in enumerate(kf):
        print('\nFold {}'.format(i))
        x_tr = x_train[train_index]
        y_tr = y[train_index]
        x_te = x_train[test_index]
        
        clf.fit(x_tr, y_tr)
        
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)
    
    m = stats.mode(oof_test_skf, axis=1)
    oof_test[:] = m[0][0]
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)       

In [7]:
# data preprocess

class DataPreprocess:
    def __init__(self, label_encoder):
        self.lbl = label_encoder
        
    def preprocess_transcation(self, transaction_df):
        categorical_columns = ['MERCHANT_CATEGORY_XCD', 'MERCHANT_CITY_NAME', 'MERCHANT_COUNTRY_XCD', 
                               'DECISION_XCD', 'TRANSACTION_CATEGORY_XCD', 'TRANSACTION_TYPE_XCD', 'SICGROUP']
        
        for col in categorical_columns:
            transaction_df[col].fillna('unknown')
            transaction_df[col] = self.lbl.fit_transform(transaction_df[col].astype(str))
        
        transaction_df = transaction_df.groupby(['ID_CPTE', 'MERCHANT_CATEGORY_XCD'])['TRANSACTION_AMT'].sum()
        transaction_df = transaction_df.reset_index()
        transaction_df = transaction_df.pivot_table('TRANSACTION_AMT', ['ID_CPTE'], 'MERCHANT_CATEGORY_XCD')
        transaction_df.columns = ['MERCHANT_CATEGORY_' + str(i) for i in transaction_df.columns]
        transaction_df = transaction_df.fillna(0)
        
        return transaction_df
    
    def preprocess_payment(self, payment_df):
        payment_df = payment_df.dropna()
        payment_df['TRANSACTION_DTTM'] = payment_df['TRANSACTION_DTTM'].apply(lambda x: str(x).split(' ')[0][:-3])
        payment_df = payment_df.sort_values(['ID_CPTE', 'TRANSACTION_DTTM'])
        payment_df['PAYMENT_N_COUNT'] = payment_df['PAYMENT_REVERSAL_XFLG'] == 'N'
        
        payment_df = payment_df.groupby(['ID_CPTE', 'TRANSACTION_DTTM'])[['TRANSACTION_AMT', 'PAYMENT_N_COUNT']].sum().reset_index()
        payment_df = payment_df.groupby('ID_CPTE').tail(12)
        
        tmp = payment_df.groupby(['ID_CPTE'])['PAYMENT_N_COUNT'].sum().reset_index()
        
        payment_df['TRANSACTION_DTTM'] = payment_df['TRANSACTION_DTTM'].apply(lambda x: x.split('-')[1])
        payment_df = payment_df.pivot_table('TRANSACTION_AMT', ['ID_CPTE'], 'TRANSACTION_DTTM')
        payment_df.columns = ['transaction_' + str(i) for i in payment_df.columns + '_month']
        payment_df = payment_df.reset_index()
        payment_df = payment_df.fillna(0)
        
        payment_df = payment_df.merge(tmp, on='ID_CPTE')
        
        return payment_df
    
    def preprocess_billing(self, billing_df):
        billing_df['PERIODID_MY'] = billing_df['PERIODID_MY'].apply(lambda x: x[:-3])
        billing_df = billing_df.sort_values(['ID_CPTE', 'PERIODID_MY'])
        billing_df = billing_df.reset_index(drop=True)
        billing_df = billing_df.groupby('ID_CPTE').tail(12)
        billing_df = billing_df.reset_index(drop=True)
        billing_df['CreditLeft'] = billing_df['CreditLimit'] - billing_df['CurrentTotalBalance']
        
        billing_df['PERIODID_MY'] = billing_df['PERIODID_MY'].apply(lambda x: x[-2:])
        credit_left = billing_df.pivot_table('CreditLeft', ['ID_CPTE'], 'PERIODID_MY')
        credit_left.columns = ['credit_left_' + str(i) for i in credit_left.columns + '_month']
        cash_balance = billing_df.pivot_table('CashBalance', ['ID_CPTE'], 'PERIODID_MY')
        cash_balance.columns = ['cash_balance_' + str(i) for i in cash_balance.columns + '_month']
        
        delq_cycle = billing_df.groupby(['ID_CPTE'])['DelqCycle'].max().reset_index()
        delq_cycle = delq_cycle.rename(columns={'DelqCycle': 'MaxDelqCycle'})
        
        credit_left = credit_left.reset_index()
        cash_balance = cash_balance.reset_index()
        
        tmp = credit_left.merge(cash_balance, on='ID_CPTE')
        tmp = tmp.merge(credit_left, on='ID_CPTE')
        tmp = tmp.merge(delq_cycle, on='ID_CPTE')
        
        return tmp
    
    def merge(self, payment, billing):
        merge_df = payment.merge(billing, on='ID_CPTE', how='right')
        return merge_df.set_index(['ID_CPTE']) 

In [8]:
billing_test.head()

,ID_CPTE,PERIODID_MY,StatementDate,CurrentTotalBalance,CashBalance,CreditLimit,DelqCycle
0,71424379,2013-11-01,2013-11-04,1444.30,0.0,3200.0,0
1,71424379,2014-05-01,2014-04-30,785.89,0.0,3200.0,0
2,71424379,2014-08-01,2014-08-02,1095.48,0.0,3200.0,0
3,71424379,2014-04-01,2014-04-02,845.30,0.0,3200.0,0
4,71424379,2013-12-01,2013-11-30,1623.28,0.0,3200.0,0


In [9]:
label_encoder = preprocessing.LabelEncoder()

In [10]:
preprocess = DataPreprocess(label_encoder)

In [11]:
processed_payment = preprocess.preprocess_payment(payment_training)

/home/data/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
processed_billing = preprocess.preprocess_billing(billing_training)

In [13]:
processed_data = preprocess.merge(processed_payment, processed_billing)

In [14]:
transaction_col = processed_data.iloc[:, :12].columns

In [15]:
# deal with missing value in payment
for col in transaction_col:
    replace_value = processed_data[processed_data[col].notna()][col].mean()
    processed_data[col] = processed_data[col].fillna(replace_value)

In [16]:
processed_data = processed_data.fillna(0)

In [17]:
processed_data = processed_data.reset_index()

In [18]:
processed_data = processed_data.merge(performance_training[['ID_CPTE', 'Default']], on='ID_CPTE')

In [19]:
processed_data = processed_data.set_index('ID_CPTE')

In [82]:
X = np.array(processed_data.iloc[:, :-1])
y = np.array(processed_data.iloc[:, -1])

In [21]:
from imblearn.over_sampling import SMOTE

In [83]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(X, y)

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
    
clf = LogisticRegression()
rf = RandomForestClassifier(min_samples_split=20, max_depth=10, random_state=0)

In [25]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [27]:
predict_clf = clf.predict(X_test)

In [28]:
roc_auc_score(predict_clf, y_test)

0.7958241477168921

In [29]:
predict_rf = rf.predict(X_test)

In [30]:
roc_auc_score(predict_rf, y_test)

0.8664037282849164

In [33]:
importance = rf.feature_importances_
importance = pd.DataFrame(importance, index=processed_data.iloc[:, :-1].columns, columns=["Importance"])
importance.sort_values('Importance', ascending=False)

,Importance
MaxDelqCycle,0.142303
credit_left_11_month_y,0.060618
cash_balance_11_month,0.055523
credit_left_11_month_x,0.050324
cash_balance_12_month,0.048794
credit_left_10_month_y,0.048088
credit_left_10_month_x,0.043153
credit_left_09_month_y,0.039432
credit_left_05_month_y,0.039132
credit_left_08_month_x,0.037565


In [58]:
# LightGBM

In [85]:
lgbm_params =  {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    #'max_depth': 15,
    'num_leaves': 270,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.75,
    'bagging_freq': 2,
    'learning_rate': 0.0175,
    'verbose': 0
}

In [86]:
lgtrain = lgb.Dataset(X_train, y_train)
lgtest = lgb.Dataset(X_test, y_test)

In [87]:
lgb_clf = lgb.train(
    lgbm_params,
    lgtrain,
    valid_sets=lgtrain,
    num_boost_round=2000,
)

[1]	training's binary_logloss: 0.683077
[2]	training's binary_logloss: 0.671411
[3]	training's binary_logloss: 0.66174
[4]	training's binary_logloss: 0.650734
[5]	training's binary_logloss: 0.640173
[6]	training's binary_logloss: 0.631496
[7]	training's binary_logloss: 0.622758
[8]	training's binary_logloss: 0.6128
[9]	training's binary_logloss: 0.604815
[10]	training's binary_logloss: 0.595508
[11]	training's binary_logloss: 0.587846
[12]	training's binary_logloss: 0.578901
[13]	training's binary_logloss: 0.571524
[14]	training's binary_logloss: 0.562966
[15]	training's binary_logloss: 0.555995
[16]	training's binary_logloss: 0.549032
[17]	training's binary_logloss: 0.541159
[18]	training's binary_logloss: 0.534781
[19]	training's binary_logloss: 0.527321
[20]	training's binary_logloss: 0.521109
[21]	training's binary_logloss: 0.51381
[22]	training's binary_logloss: 0.508149
[23]	training's binary_logloss: 0.502265
[24]	training's binary_logloss: 0.496644
[25]	training's binary_loglos

[218]	training's binary_logloss: 0.102053
[219]	training's binary_logloss: 0.101424
[220]	training's binary_logloss: 0.100735
[221]	training's binary_logloss: 0.100059
[222]	training's binary_logloss: 0.0993451
[223]	training's binary_logloss: 0.0986804
[224]	training's binary_logloss: 0.0980405
[225]	training's binary_logloss: 0.0973536
[226]	training's binary_logloss: 0.0967242
[227]	training's binary_logloss: 0.0960844
[228]	training's binary_logloss: 0.095445
[229]	training's binary_logloss: 0.094756
[230]	training's binary_logloss: 0.0941217
[231]	training's binary_logloss: 0.0934661
[232]	training's binary_logloss: 0.0928483
[233]	training's binary_logloss: 0.0922116
[234]	training's binary_logloss: 0.0915924
[235]	training's binary_logloss: 0.0909849
[236]	training's binary_logloss: 0.0904179
[237]	training's binary_logloss: 0.0898288
[238]	training's binary_logloss: 0.0892371
[239]	training's binary_logloss: 0.0886338
[240]	training's binary_logloss: 0.0880039
[241]	training's 

[412]	training's binary_logloss: 0.0295305
[413]	training's binary_logloss: 0.029345
[414]	training's binary_logloss: 0.0291647
[415]	training's binary_logloss: 0.0289837
[416]	training's binary_logloss: 0.0287971
[417]	training's binary_logloss: 0.0286154
[418]	training's binary_logloss: 0.0284294
[419]	training's binary_logloss: 0.0282546
[420]	training's binary_logloss: 0.0280769
[421]	training's binary_logloss: 0.0278981
[422]	training's binary_logloss: 0.027736
[423]	training's binary_logloss: 0.0275564
[424]	training's binary_logloss: 0.0273756
[425]	training's binary_logloss: 0.027217
[426]	training's binary_logloss: 0.027059
[427]	training's binary_logloss: 0.0268799
[428]	training's binary_logloss: 0.0267065
[429]	training's binary_logloss: 0.0265294
[430]	training's binary_logloss: 0.0263618
[431]	training's binary_logloss: 0.0262007
[432]	training's binary_logloss: 0.0260418
[433]	training's binary_logloss: 0.0258821
[434]	training's binary_logloss: 0.0257228
[435]	training'

[617]	training's binary_logloss: 0.00842727
[618]	training's binary_logloss: 0.00837541
[619]	training's binary_logloss: 0.00832656
[620]	training's binary_logloss: 0.00827878
[621]	training's binary_logloss: 0.00822958
[622]	training's binary_logloss: 0.00817862
[623]	training's binary_logloss: 0.00812894
[624]	training's binary_logloss: 0.00807813
[625]	training's binary_logloss: 0.00803396
[626]	training's binary_logloss: 0.00798565
[627]	training's binary_logloss: 0.00793998
[628]	training's binary_logloss: 0.00789414
[629]	training's binary_logloss: 0.00784636
[630]	training's binary_logloss: 0.00780151
[631]	training's binary_logloss: 0.00775369
[632]	training's binary_logloss: 0.0077066
[633]	training's binary_logloss: 0.00766232
[634]	training's binary_logloss: 0.00761586
[635]	training's binary_logloss: 0.00756881
[636]	training's binary_logloss: 0.00752483
[637]	training's binary_logloss: 0.00747906
[638]	training's binary_logloss: 0.00743348
[639]	training's binary_logloss: 

[817]	training's binary_logloss: 0.00254672
[818]	training's binary_logloss: 0.00253204
[819]	training's binary_logloss: 0.0025174
[820]	training's binary_logloss: 0.00250264
[821]	training's binary_logloss: 0.00248757
[822]	training's binary_logloss: 0.00247268
[823]	training's binary_logloss: 0.00245778
[824]	training's binary_logloss: 0.00244443
[825]	training's binary_logloss: 0.00242947
[826]	training's binary_logloss: 0.0024154
[827]	training's binary_logloss: 0.00240022
[828]	training's binary_logloss: 0.00238605
[829]	training's binary_logloss: 0.00237219
[830]	training's binary_logloss: 0.00235766
[831]	training's binary_logloss: 0.00234338
[832]	training's binary_logloss: 0.00233039
[833]	training's binary_logloss: 0.00231598
[834]	training's binary_logloss: 0.0023025
[835]	training's binary_logloss: 0.0022881
[836]	training's binary_logloss: 0.00227462
[837]	training's binary_logloss: 0.00226126
[838]	training's binary_logloss: 0.00224796
[839]	training's binary_logloss: 0.0

...

[1192]	training's binary_logloss: 0.000278291
[1193]	training's binary_logloss: 0.000276695
[1194]	training's binary_logloss: 0.000275051
[1195]	training's binary_logloss: 0.000273424
[1196]	training's binary_logloss: 0.000271729
[1197]	training's binary_logloss: 0.000270088
[1198]	training's binary_logloss: 0.000268421
[1199]	training's binary_logloss: 0.000266774
[1200]	training's binary_logloss: 0.000265126
[1201]	training's binary_logloss: 0.000263619
[1202]	training's binary_logloss: 0.000262012
[1203]	training's binary_logloss: 0.000260499
[1204]	training's binary_logloss: 0.000258967
[1205]	training's binary_logloss: 0.000257338
[1206]	training's binary_logloss: 0.000255799
[1207]	training's binary_logloss: 0.000254258
[1208]	training's binary_logloss: 0.000252685
[1209]	training's binary_logloss: 0.000251281
[1210]	training's binary_logloss: 0.00024978
[1211]	training's binary_logloss: 0.000248275
[1212]	training's binary_logloss: 0.000246813
[1213]	training's binary_logloss: 0

[1384]	training's binary_logloss: 9.02755e-05
[1385]	training's binary_logloss: 8.97708e-05
[1386]	training's binary_logloss: 8.92519e-05
[1387]	training's binary_logloss: 8.87429e-05
[1388]	training's binary_logloss: 8.82255e-05
[1389]	training's binary_logloss: 8.76962e-05
[1390]	training's binary_logloss: 8.71702e-05
[1391]	training's binary_logloss: 8.6669e-05
[1392]	training's binary_logloss: 8.61805e-05
[1393]	training's binary_logloss: 8.57042e-05
[1394]	training's binary_logloss: 8.52185e-05
[1395]	training's binary_logloss: 8.4712e-05
[1396]	training's binary_logloss: 8.41874e-05
[1397]	training's binary_logloss: 8.37538e-05
[1398]	training's binary_logloss: 8.328e-05
[1399]	training's binary_logloss: 8.28104e-05
[1400]	training's binary_logloss: 8.23523e-05
[1401]	training's binary_logloss: 8.18948e-05
[1402]	training's binary_logloss: 8.14314e-05
[1403]	training's binary_logloss: 8.09409e-05
[1404]	training's binary_logloss: 8.04714e-05
[1405]	training's binary_logloss: 8.00

[1576]	training's binary_logloss: 3.19385e-05
[1577]	training's binary_logloss: 3.17816e-05
[1578]	training's binary_logloss: 3.16355e-05
[1579]	training's binary_logloss: 3.14915e-05
[1580]	training's binary_logloss: 3.13475e-05
[1581]	training's binary_logloss: 3.12063e-05
[1582]	training's binary_logloss: 3.10592e-05
[1583]	training's binary_logloss: 3.09125e-05
[1584]	training's binary_logloss: 3.07677e-05
[1585]	training's binary_logloss: 3.06154e-05
[1586]	training's binary_logloss: 3.04732e-05
[1587]	training's binary_logloss: 3.03375e-05
[1588]	training's binary_logloss: 3.0202e-05
[1589]	training's binary_logloss: 3.00644e-05
[1590]	training's binary_logloss: 2.99147e-05
[1591]	training's binary_logloss: 2.97747e-05
[1592]	training's binary_logloss: 2.96426e-05
[1593]	training's binary_logloss: 2.95014e-05
[1594]	training's binary_logloss: 2.93854e-05
[1595]	training's binary_logloss: 2.92503e-05
[1596]	training's binary_logloss: 2.91143e-05
[1597]	training's binary_logloss: 2

[1762]	training's binary_logloss: 1.60471e-05
[1763]	training's binary_logloss: 1.59985e-05
[1764]	training's binary_logloss: 1.59479e-05
[1765]	training's binary_logloss: 1.59049e-05
[1766]	training's binary_logloss: 1.58632e-05
[1767]	training's binary_logloss: 1.58185e-05
[1768]	training's binary_logloss: 1.57755e-05
[1769]	training's binary_logloss: 1.57349e-05
[1770]	training's binary_logloss: 1.56923e-05
[1771]	training's binary_logloss: 1.56472e-05
[1772]	training's binary_logloss: 1.56056e-05
[1773]	training's binary_logloss: 1.55642e-05
[1774]	training's binary_logloss: 1.55179e-05
[1775]	training's binary_logloss: 1.54739e-05
[1776]	training's binary_logloss: 1.54312e-05
[1777]	training's binary_logloss: 1.53888e-05
[1778]	training's binary_logloss: 1.53482e-05
[1779]	training's binary_logloss: 1.53068e-05
[1780]	training's binary_logloss: 1.52672e-05
[1781]	training's binary_logloss: 1.52276e-05
[1782]	training's binary_logloss: 1.51821e-05
[1783]	training's binary_logloss: 

[1961]	training's binary_logloss: 1.03318e-05
[1962]	training's binary_logloss: 1.03117e-05
[1963]	training's binary_logloss: 1.02954e-05
[1964]	training's binary_logloss: 1.02756e-05
[1965]	training's binary_logloss: 1.02575e-05
[1966]	training's binary_logloss: 1.02397e-05
[1967]	training's binary_logloss: 1.02231e-05
[1968]	training's binary_logloss: 1.02042e-05
[1969]	training's binary_logloss: 1.01859e-05
[1970]	training's binary_logloss: 1.017e-05
[1971]	training's binary_logloss: 1.01533e-05
[1972]	training's binary_logloss: 1.01356e-05
[1973]	training's binary_logloss: 1.0117e-05
[1974]	training's binary_logloss: 1.01015e-05
[1975]	training's binary_logloss: 1.00852e-05
[1976]	training's binary_logloss: 1.00677e-05
[1977]	training's binary_logloss: 1.00497e-05
[1978]	training's binary_logloss: 1.00306e-05
[1979]	training's binary_logloss: 1.00138e-05
[1980]	training's binary_logloss: 9.99747e-06
[1981]	training's binary_logloss: 9.98154e-06
[1982]	training's binary_logloss: 9.9

In [88]:
predict_lgb = lgb_clf.predict(X_test)

In [89]:
predict_lgb = np.array([0 if i < 0.6 else 1 for i in predict_lgb])

In [90]:
roc_auc_score(predict_lgb, y_test)

0.9236584269933988

### Grid Search, tune parameters

In [55]:
# Create parameters to search
gridParams = {
    'learning_rate': [0.0125, 0.0175, 0.0225],
    'n_estimators': [40],
    'num_leaves': [170, 220, 270, 320],
    #'max_depth': [15, 25, 35],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'feature_fraction': [0.4, 0.5, 0.6]
    }

In [56]:
# Create classifier to use. Note that parameters have to be input manually
# not as a dict!
mdl = lgb.LGBMClassifier(boosting_type= 'gbdt', objective = 'binary')

In [57]:
grid = GridSearchCV(mdl, gridParams, verbose=0, cv=5, n_jobs=2)

In [58]:
grid.fit(X_res, y_res)

/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

...

/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

GridSearchCV(cv=5, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'learning_rate': [0.0125, 0.0175, 0.0225], 'n_estimators': [40], 'num_leaves': [170, 220, 270, 320], 'boosting_type': ['gbdt'], 'objective': ['binary'], 'feature_fraction': [0.4, 0.5, 0.6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [59]:
# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

{'boosting_type': 'gbdt', 'feature_fraction': 0.5, 'learning_rate': 0.0225, 'n_estimators': 40, 'num_leaves': 320, 'objective': 'binary'}
0.8844150432336702


In [91]:
# Using parameters already set above, replace in the best from the grid search

# params['max_bin'] = grid.best_params_['max_bin']
lgbm_params['feature_fraction'] = grid.best_params_['feature_fraction']
lgbm_params['learning_rate'] = grid.best_params_['learning_rate']
lgbm_params['num_leaves'] = grid.best_params_['num_leaves']
#lgbm_params['max_depth'] = grid.best_params_['max_depth']
#lgbm_params['reg_alpha'] = grid.best_params_['reg_alpha']
#lgbm_params['reg_lambda'] = grid.best_params_['reg_lambda']

In [70]:
print('Fitting with params: ')
print(lgbm_params)

Fitting with params: 
{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 'num_leaves': 320, 'feature_fraction': 0.5, 'bagging_fraction': 0.75, 'bagging_freq': 2, 'learning_rate': 0.0225, 'verbose': 0}


In [92]:
lgtrain = lgb.Dataset(X_train, y_train)
lgtest = lgb.Dataset(X_test, y_test)

In [93]:
lgb_clf = lgb.train(
    lgbm_params,
    lgtrain,
    valid_sets=lgtrain,
    num_boost_round=2000
)

[1]	training's binary_logloss: 0.680222
[2]	training's binary_logloss: 0.665389
[3]	training's binary_logloss: 0.653191
[4]	training's binary_logloss: 0.639386
[5]	training's binary_logloss: 0.62627
[6]	training's binary_logloss: 0.615549
[7]	training's binary_logloss: 0.604853
[8]	training's binary_logloss: 0.59271
[9]	training's binary_logloss: 0.583045
[10]	training's binary_logloss: 0.571847
[11]	training's binary_logloss: 0.562659
[12]	training's binary_logloss: 0.552023
[13]	training's binary_logloss: 0.543252
[14]	training's binary_logloss: 0.5332
[15]	training's binary_logloss: 0.525082
[16]	training's binary_logloss: 0.516985
[17]	training's binary_logloss: 0.507895
[18]	training's binary_logloss: 0.500583
[19]	training's binary_logloss: 0.492051
[20]	training's binary_logloss: 0.485031
[21]	training's binary_logloss: 0.476843
[22]	training's binary_logloss: 0.470439
[23]	training's binary_logloss: 0.463842
[24]	training's binary_logloss: 0.457568
[25]	training's binary_loglos

[210]	training's binary_logloss: 0.0718775
[211]	training's binary_logloss: 0.0712573
[212]	training's binary_logloss: 0.0706457
[213]	training's binary_logloss: 0.0700746
[214]	training's binary_logloss: 0.0694923
[215]	training's binary_logloss: 0.0689093
[216]	training's binary_logloss: 0.0683074
[217]	training's binary_logloss: 0.0677445
[218]	training's binary_logloss: 0.0671402
[219]	training's binary_logloss: 0.0665853
[220]	training's binary_logloss: 0.0660362
[221]	training's binary_logloss: 0.0654867
[222]	training's binary_logloss: 0.0649203
[223]	training's binary_logloss: 0.0643821
[224]	training's binary_logloss: 0.0638353
[225]	training's binary_logloss: 0.0633037
[226]	training's binary_logloss: 0.0627686
[227]	training's binary_logloss: 0.062247
[228]	training's binary_logloss: 0.0616928
[229]	training's binary_logloss: 0.0611645
[230]	training's binary_logloss: 0.060674
[231]	training's binary_logloss: 0.060146
[232]	training's binary_logloss: 0.0596419
[233]	training

[407]	training's binary_logloss: 0.0142639
[408]	training's binary_logloss: 0.0141573
[409]	training's binary_logloss: 0.0140331
[410]	training's binary_logloss: 0.0139167
[411]	training's binary_logloss: 0.013798
[412]	training's binary_logloss: 0.013688
[413]	training's binary_logloss: 0.0135787
[414]	training's binary_logloss: 0.0134703
[415]	training's binary_logloss: 0.0133609
[416]	training's binary_logloss: 0.0132512
[417]	training's binary_logloss: 0.013148
[418]	training's binary_logloss: 0.0130366
[419]	training's binary_logloss: 0.0129306
[420]	training's binary_logloss: 0.0128261
[421]	training's binary_logloss: 0.0127246
[422]	training's binary_logloss: 0.0126299
[423]	training's binary_logloss: 0.0125329
[424]	training's binary_logloss: 0.0124376
[425]	training's binary_logloss: 0.0123354
[426]	training's binary_logloss: 0.0122491
[427]	training's binary_logloss: 0.0121527
[428]	training's binary_logloss: 0.0120527
[429]	training's binary_logloss: 0.0119477
[430]	training

[604]	training's binary_logloss: 0.00299644
[605]	training's binary_logloss: 0.00297256
[606]	training's binary_logloss: 0.00294822
[607]	training's binary_logloss: 0.00292443
[608]	training's binary_logloss: 0.0029017
[609]	training's binary_logloss: 0.00288082
[610]	training's binary_logloss: 0.0028574
[611]	training's binary_logloss: 0.00283461
[612]	training's binary_logloss: 0.00281261
[613]	training's binary_logloss: 0.00279116
[614]	training's binary_logloss: 0.00276892
[615]	training's binary_logloss: 0.00274794
[616]	training's binary_logloss: 0.00272591
[617]	training's binary_logloss: 0.00270472
[618]	training's binary_logloss: 0.00268229
[619]	training's binary_logloss: 0.00266158
[620]	training's binary_logloss: 0.00264038
[621]	training's binary_logloss: 0.00261928
[622]	training's binary_logloss: 0.00259988
[623]	training's binary_logloss: 0.00257971
[624]	training's binary_logloss: 0.00255949
[625]	training's binary_logloss: 0.00254063
[626]	training's binary_logloss: 0

[796]	training's binary_logloss: 0.000665436
[797]	training's binary_logloss: 0.000660075
[798]	training's binary_logloss: 0.00065479
[799]	training's binary_logloss: 0.000649751
[800]	training's binary_logloss: 0.000644324
[801]	training's binary_logloss: 0.00063918
[802]	training's binary_logloss: 0.000633717
[803]	training's binary_logloss: 0.000628787
[804]	training's binary_logloss: 0.00062438
[805]	training's binary_logloss: 0.000619466
[806]	training's binary_logloss: 0.000614507
[807]	training's binary_logloss: 0.000609592
[808]	training's binary_logloss: 0.000605237
[809]	training's binary_logloss: 0.000600439
[810]	training's binary_logloss: 0.000595942
[811]	training's binary_logloss: 0.000591371
[812]	training's binary_logloss: 0.000587394
[813]	training's binary_logloss: 0.000582484
[814]	training's binary_logloss: 0.000578136
[815]	training's binary_logloss: 0.000573411
[816]	training's binary_logloss: 0.000568867
[817]	training's binary_logloss: 0.000564542
[818]	trainin

...

[1164]	training's binary_logloss: 4.16505e-05
[1165]	training's binary_logloss: 4.13743e-05
[1166]	training's binary_logloss: 4.10963e-05
[1167]	training's binary_logloss: 4.08226e-05
[1168]	training's binary_logloss: 4.05317e-05
[1169]	training's binary_logloss: 4.02618e-05
[1170]	training's binary_logloss: 3.99749e-05
[1171]	training's binary_logloss: 3.97036e-05
[1172]	training's binary_logloss: 3.94376e-05
[1173]	training's binary_logloss: 3.91676e-05
[1174]	training's binary_logloss: 3.89108e-05
[1175]	training's binary_logloss: 3.86341e-05
[1176]	training's binary_logloss: 3.83648e-05
[1177]	training's binary_logloss: 3.81288e-05
[1178]	training's binary_logloss: 3.78801e-05
[1179]	training's binary_logloss: 3.76234e-05
[1180]	training's binary_logloss: 3.73545e-05
[1181]	training's binary_logloss: 3.71047e-05
[1182]	training's binary_logloss: 3.68688e-05
[1183]	training's binary_logloss: 3.66272e-05
[1184]	training's binary_logloss: 3.63637e-05
[1185]	training's binary_logloss: 

[1344]	training's binary_logloss: 1.62183e-05
[1345]	training's binary_logloss: 1.6161e-05
[1346]	training's binary_logloss: 1.61064e-05
[1347]	training's binary_logloss: 1.605e-05
[1348]	training's binary_logloss: 1.59934e-05
[1349]	training's binary_logloss: 1.59328e-05
[1350]	training's binary_logloss: 1.58742e-05
[1351]	training's binary_logloss: 1.58222e-05
[1352]	training's binary_logloss: 1.57637e-05
[1353]	training's binary_logloss: 1.57048e-05
[1354]	training's binary_logloss: 1.56482e-05
[1355]	training's binary_logloss: 1.55935e-05
[1356]	training's binary_logloss: 1.55417e-05
[1357]	training's binary_logloss: 1.54888e-05
[1358]	training's binary_logloss: 1.54332e-05
[1359]	training's binary_logloss: 1.53762e-05
[1360]	training's binary_logloss: 1.53159e-05
[1361]	training's binary_logloss: 1.52665e-05
[1362]	training's binary_logloss: 1.52178e-05
[1363]	training's binary_logloss: 1.51688e-05
[1364]	training's binary_logloss: 1.51197e-05
[1365]	training's binary_logloss: 1.5

[1553]	training's binary_logloss: 9.25781e-06
[1554]	training's binary_logloss: 9.23756e-06
[1555]	training's binary_logloss: 9.22115e-06
[1556]	training's binary_logloss: 9.20403e-06
[1557]	training's binary_logloss: 9.18673e-06
[1558]	training's binary_logloss: 9.16882e-06
[1559]	training's binary_logloss: 9.15068e-06
[1560]	training's binary_logloss: 9.13266e-06
[1561]	training's binary_logloss: 9.11568e-06
[1562]	training's binary_logloss: 9.0989e-06
[1563]	training's binary_logloss: 9.07901e-06
[1564]	training's binary_logloss: 9.06201e-06
[1565]	training's binary_logloss: 9.0431e-06
[1566]	training's binary_logloss: 9.02718e-06
[1567]	training's binary_logloss: 9.01013e-06
[1568]	training's binary_logloss: 8.99377e-06
[1569]	training's binary_logloss: 8.97788e-06
[1570]	training's binary_logloss: 8.96065e-06
[1571]	training's binary_logloss: 8.94516e-06
[1572]	training's binary_logloss: 8.92878e-06
[1573]	training's binary_logloss: 8.91231e-06
[1574]	training's binary_logloss: 8.

[1772]	training's binary_logloss: 6.53492e-06
[1773]	training's binary_logloss: 6.52657e-06
[1774]	training's binary_logloss: 6.51756e-06
[1775]	training's binary_logloss: 6.50965e-06
[1776]	training's binary_logloss: 6.50126e-06
[1777]	training's binary_logloss: 6.49219e-06
[1778]	training's binary_logloss: 6.48339e-06
[1779]	training's binary_logloss: 6.47467e-06
[1780]	training's binary_logloss: 6.46675e-06
[1781]	training's binary_logloss: 6.45843e-06
[1782]	training's binary_logloss: 6.44942e-06
[1783]	training's binary_logloss: 6.44213e-06
[1784]	training's binary_logloss: 6.43393e-06
[1785]	training's binary_logloss: 6.42529e-06
[1786]	training's binary_logloss: 6.41752e-06
[1787]	training's binary_logloss: 6.40996e-06
[1788]	training's binary_logloss: 6.40191e-06
[1789]	training's binary_logloss: 6.39398e-06
[1790]	training's binary_logloss: 6.38634e-06
[1791]	training's binary_logloss: 6.37877e-06
[1792]	training's binary_logloss: 6.37166e-06
[1793]	training's binary_logloss: 

[1972]	training's binary_logloss: 5.24024e-06
[1973]	training's binary_logloss: 5.23445e-06
[1974]	training's binary_logloss: 5.2301e-06
[1975]	training's binary_logloss: 5.22441e-06
[1976]	training's binary_logloss: 5.21882e-06
[1977]	training's binary_logloss: 5.2137e-06
[1978]	training's binary_logloss: 5.2088e-06
[1979]	training's binary_logloss: 5.20424e-06
[1980]	training's binary_logloss: 5.19881e-06
[1981]	training's binary_logloss: 5.19479e-06
[1982]	training's binary_logloss: 5.18965e-06
[1983]	training's binary_logloss: 5.18484e-06
[1984]	training's binary_logloss: 5.18018e-06
[1985]	training's binary_logloss: 5.17597e-06
[1986]	training's binary_logloss: 5.17166e-06
[1987]	training's binary_logloss: 5.16702e-06
[1988]	training's binary_logloss: 5.16228e-06
[1989]	training's binary_logloss: 5.15773e-06
[1990]	training's binary_logloss: 5.15253e-06
[1991]	training's binary_logloss: 5.1477e-06
[1992]	training's binary_logloss: 5.14276e-06
[1993]	training's binary_logloss: 5.13

In [95]:
predict_lgb = lgb_clf.predict(X_test)
predict_lgb = np.array([0 if i < 0.6 else 1 for i in predict_lgb])
roc_auc_score(predict_lgb, y_test)

0.9227782792854905

In [32]:
from sklearn.externals import joblib
joblib.dump(clf, 'random_forest.pkl')

['random_forest.pkl']

In [96]:
# deploy model
lgtrain = lgb.Dataset(X_res, y_res)

final_lgb = lgb.train(
    lgbm_params,
    lgtrain,
    valid_sets=lgtrain,
    num_boost_round=2000
)

[1]	training's binary_logloss: 0.679963
[2]	training's binary_logloss: 0.66497
[3]	training's binary_logloss: 0.652721
[4]	training's binary_logloss: 0.638789
[5]	training's binary_logloss: 0.625434
[6]	training's binary_logloss: 0.61472
[7]	training's binary_logloss: 0.603808
[8]	training's binary_logloss: 0.591721
[9]	training's binary_logloss: 0.581847
[10]	training's binary_logloss: 0.57042
[11]	training's binary_logloss: 0.561189
[12]	training's binary_logloss: 0.550441
[13]	training's binary_logloss: 0.541623
[14]	training's binary_logloss: 0.53153
[15]	training's binary_logloss: 0.52316
[16]	training's binary_logloss: 0.515254
[17]	training's binary_logloss: 0.505888
[18]	training's binary_logloss: 0.498422
[19]	training's binary_logloss: 0.489725
[20]	training's binary_logloss: 0.482526
[21]	training's binary_logloss: 0.47419
[22]	training's binary_logloss: 0.467509
[23]	training's binary_logloss: 0.460875
[24]	training's binary_logloss: 0.454472
[25]	training's binary_logloss:

[199]	training's binary_logloss: 0.0772638
[200]	training's binary_logloss: 0.0766379
[201]	training's binary_logloss: 0.0759774
[202]	training's binary_logloss: 0.0753103
[203]	training's binary_logloss: 0.0747256
[204]	training's binary_logloss: 0.0741488
[205]	training's binary_logloss: 0.0734999
[206]	training's binary_logloss: 0.0729097
[207]	training's binary_logloss: 0.07227
[208]	training's binary_logloss: 0.0715998
[209]	training's binary_logloss: 0.0709609
[210]	training's binary_logloss: 0.0703132
[211]	training's binary_logloss: 0.0697548
[212]	training's binary_logloss: 0.0691784
[213]	training's binary_logloss: 0.0685933
[214]	training's binary_logloss: 0.0680614
[215]	training's binary_logloss: 0.0674941
[216]	training's binary_logloss: 0.0669178
[217]	training's binary_logloss: 0.066328
[218]	training's binary_logloss: 0.0658082
[219]	training's binary_logloss: 0.0653198
[220]	training's binary_logloss: 0.0647656
[221]	training's binary_logloss: 0.0642212
[222]	training

[396]	training's binary_logloss: 0.0163086
[397]	training's binary_logloss: 0.0161884
[398]	training's binary_logloss: 0.0160625
[399]	training's binary_logloss: 0.0159455
[400]	training's binary_logloss: 0.0158206
[401]	training's binary_logloss: 0.0157
[402]	training's binary_logloss: 0.0155789
[403]	training's binary_logloss: 0.0154607
[404]	training's binary_logloss: 0.0153426
[405]	training's binary_logloss: 0.0152259
[406]	training's binary_logloss: 0.0151091
[407]	training's binary_logloss: 0.0149929
[408]	training's binary_logloss: 0.0148796
[409]	training's binary_logloss: 0.0147623
[410]	training's binary_logloss: 0.0146502
[411]	training's binary_logloss: 0.0145413
[412]	training's binary_logloss: 0.0144362
[413]	training's binary_logloss: 0.0143286
[414]	training's binary_logloss: 0.0142198
[415]	training's binary_logloss: 0.0141239
[416]	training's binary_logloss: 0.0140186
[417]	training's binary_logloss: 0.0139128
[418]	training's binary_logloss: 0.0138017
[419]	training

[586]	training's binary_logloss: 0.00391371
[587]	training's binary_logloss: 0.00388607
[588]	training's binary_logloss: 0.00385704
[589]	training's binary_logloss: 0.00382918
[590]	training's binary_logloss: 0.0038032
[591]	training's binary_logloss: 0.00377537
[592]	training's binary_logloss: 0.00374882
[593]	training's binary_logloss: 0.00372338
[594]	training's binary_logloss: 0.00369763
[595]	training's binary_logloss: 0.00367219
[596]	training's binary_logloss: 0.00364397
[597]	training's binary_logloss: 0.00361821
[598]	training's binary_logloss: 0.00359339
[599]	training's binary_logloss: 0.0035683
[600]	training's binary_logloss: 0.00354372
[601]	training's binary_logloss: 0.00351886
[602]	training's binary_logloss: 0.00349407
[603]	training's binary_logloss: 0.00346782
[604]	training's binary_logloss: 0.00344381
[605]	training's binary_logloss: 0.0034181
[606]	training's binary_logloss: 0.0033936
[607]	training's binary_logloss: 0.00337161
[608]	training's binary_logloss: 0.0

[778]	training's binary_logloss: 0.000917744
[779]	training's binary_logloss: 0.000911321
[780]	training's binary_logloss: 0.000904474
[781]	training's binary_logloss: 0.000897863
[782]	training's binary_logloss: 0.000891265
[783]	training's binary_logloss: 0.000884333
[784]	training's binary_logloss: 0.000877636
[785]	training's binary_logloss: 0.000871198
[786]	training's binary_logloss: 0.000864699
[787]	training's binary_logloss: 0.000857901
[788]	training's binary_logloss: 0.000851647
[789]	training's binary_logloss: 0.000845216
[790]	training's binary_logloss: 0.000838978
[791]	training's binary_logloss: 0.000832331
[792]	training's binary_logloss: 0.000826494
[793]	training's binary_logloss: 0.000820733
[794]	training's binary_logloss: 0.000814784
[795]	training's binary_logloss: 0.000808437
[796]	training's binary_logloss: 0.000802418
[797]	training's binary_logloss: 0.000796565
[798]	training's binary_logloss: 0.000790911
[799]	training's binary_logloss: 0.000784638
[800]	trai

...

[1151]	training's binary_logloss: 7.28559e-05
[1152]	training's binary_logloss: 7.2054e-05
[1153]	training's binary_logloss: 7.12271e-05
[1154]	training's binary_logloss: 7.04427e-05
[1155]	training's binary_logloss: 6.96573e-05
[1156]	training's binary_logloss: 6.8886e-05
[1157]	training's binary_logloss: 6.81261e-05
[1158]	training's binary_logloss: 6.74114e-05
[1159]	training's binary_logloss: 6.67075e-05
[1160]	training's binary_logloss: 6.60037e-05
[1161]	training's binary_logloss: 6.53071e-05
[1162]	training's binary_logloss: 6.46151e-05
[1163]	training's binary_logloss: 6.39462e-05
[1164]	training's binary_logloss: 6.32633e-05
[1165]	training's binary_logloss: 6.25969e-05
[1166]	training's binary_logloss: 6.19462e-05
[1167]	training's binary_logloss: 6.13293e-05
[1168]	training's binary_logloss: 6.06975e-05
[1169]	training's binary_logloss: 6.00733e-05
[1170]	training's binary_logloss: 5.94693e-05
[1171]	training's binary_logloss: 5.88608e-05
[1172]	training's binary_logloss: 5.

[1338]	training's binary_logloss: 1.86435e-05
[1339]	training's binary_logloss: 1.85559e-05
[1340]	training's binary_logloss: 1.84692e-05
[1341]	training's binary_logloss: 1.83826e-05
[1342]	training's binary_logloss: 1.83039e-05
[1343]	training's binary_logloss: 1.82201e-05
[1344]	training's binary_logloss: 1.81407e-05
[1345]	training's binary_logloss: 1.80676e-05
[1346]	training's binary_logloss: 1.79917e-05
[1347]	training's binary_logloss: 1.79126e-05
[1348]	training's binary_logloss: 1.7833e-05
[1349]	training's binary_logloss: 1.77569e-05
[1350]	training's binary_logloss: 1.7679e-05
[1351]	training's binary_logloss: 1.75983e-05
[1352]	training's binary_logloss: 1.75279e-05
[1353]	training's binary_logloss: 1.74535e-05
[1354]	training's binary_logloss: 1.73776e-05
[1355]	training's binary_logloss: 1.7304e-05
[1356]	training's binary_logloss: 1.7228e-05
[1357]	training's binary_logloss: 1.71516e-05
[1358]	training's binary_logloss: 1.70838e-05
[1359]	training's binary_logloss: 1.70

[1523]	training's binary_logloss: 1.01089e-05
[1524]	training's binary_logloss: 1.00826e-05
[1525]	training's binary_logloss: 1.00551e-05
[1526]	training's binary_logloss: 1.0032e-05
[1527]	training's binary_logloss: 1.00089e-05
[1528]	training's binary_logloss: 9.98494e-06
[1529]	training's binary_logloss: 9.961e-06
[1530]	training's binary_logloss: 9.93624e-06
[1531]	training's binary_logloss: 9.91411e-06
[1532]	training's binary_logloss: 9.89048e-06
[1533]	training's binary_logloss: 9.86731e-06
[1534]	training's binary_logloss: 9.84421e-06
[1535]	training's binary_logloss: 9.82192e-06
[1536]	training's binary_logloss: 9.79646e-06
[1537]	training's binary_logloss: 9.77435e-06
[1538]	training's binary_logloss: 9.75123e-06
[1539]	training's binary_logloss: 9.72864e-06
[1540]	training's binary_logloss: 9.70609e-06
[1541]	training's binary_logloss: 9.68156e-06
[1542]	training's binary_logloss: 9.65961e-06
[1543]	training's binary_logloss: 9.63833e-06
[1544]	training's binary_logloss: 9.6

[1703]	training's binary_logloss: 7.0865e-06
[1704]	training's binary_logloss: 7.07601e-06
[1705]	training's binary_logloss: 7.06373e-06
[1706]	training's binary_logloss: 7.05289e-06
[1707]	training's binary_logloss: 7.04251e-06
[1708]	training's binary_logloss: 7.03189e-06
[1709]	training's binary_logloss: 7.02024e-06
[1710]	training's binary_logloss: 7.00908e-06
[1711]	training's binary_logloss: 6.99807e-06
[1712]	training's binary_logloss: 6.98759e-06
[1713]	training's binary_logloss: 6.97678e-06
[1714]	training's binary_logloss: 6.96561e-06
[1715]	training's binary_logloss: 6.95475e-06
[1716]	training's binary_logloss: 6.94468e-06
[1717]	training's binary_logloss: 6.93301e-06
[1718]	training's binary_logloss: 6.92261e-06
[1719]	training's binary_logloss: 6.91172e-06
[1720]	training's binary_logloss: 6.90023e-06
[1721]	training's binary_logloss: 6.8906e-06
[1722]	training's binary_logloss: 6.88054e-06
[1723]	training's binary_logloss: 6.8698e-06
[1724]	training's binary_logloss: 6.8

[1885]	training's binary_logloss: 5.52485e-06
[1886]	training's binary_logloss: 5.51804e-06
[1887]	training's binary_logloss: 5.51168e-06
[1888]	training's binary_logloss: 5.50587e-06
[1889]	training's binary_logloss: 5.49901e-06
[1890]	training's binary_logloss: 5.49267e-06
[1891]	training's binary_logloss: 5.48657e-06
[1892]	training's binary_logloss: 5.47892e-06
[1893]	training's binary_logloss: 5.47216e-06
[1894]	training's binary_logloss: 5.466e-06
[1895]	training's binary_logloss: 5.4598e-06
[1896]	training's binary_logloss: 5.45356e-06
[1897]	training's binary_logloss: 5.44636e-06
[1898]	training's binary_logloss: 5.43952e-06
[1899]	training's binary_logloss: 5.43297e-06
[1900]	training's binary_logloss: 5.42741e-06
[1901]	training's binary_logloss: 5.42125e-06
[1902]	training's binary_logloss: 5.4146e-06
[1903]	training's binary_logloss: 5.40882e-06
[1904]	training's binary_logloss: 5.40225e-06
[1905]	training's binary_logloss: 5.39626e-06
[1906]	training's binary_logloss: 5.39

### Fit the final model to the testing data

In [97]:
processed_payment_test = preprocess.preprocess_payment(payment_test)
processed_billing_test = preprocess.preprocess_billing(billing_test)

/home/data/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [98]:
processed_test = preprocess.merge(processed_payment_test, processed_billing_test)

In [99]:
for col in transaction_col:
    replace_value = processed_test[processed_test[col].notna()][col].mean()
    processed_test[col] = processed_test[col].fillna(replace_value)

In [100]:
processed_test = processed_test.fillna(0)

In [101]:
processed_test.head()

,transaction_01_month,transaction_02_month,transaction_03_month,transaction_04_month,transaction_05_month,transaction_06_month,transaction_07_month,transaction_08_month,transaction_09_month,transaction_10_month,...,credit_left_04_month_y,credit_left_05_month_y,credit_left_06_month_y,credit_left_07_month_y,credit_left_08_month_y,credit_left_09_month_y,credit_left_10_month_y,credit_left_11_month_y,credit_left_12_month_y,MaxDelqCycle
ID_CPTE,,,,,,,,,,,,,,,,,,,,,
10074849,411.00,340.26,993.92,0.0,906.38,363.00,915.54,609.50,626.85,396.93,...,1082.89,1689.60,1355.5,1514.89,1691.76,1303.07,1254.59,1586.58,1237.53,0
10086539,0.00,556.92,832.00,642.0,661.26,1880.00,950.86,1591.00,1048.60,500.00,...,932.32,611.21,192.4,1309.15,845.20,601.42,394.90,866.74,865.76,0
10140908,214.10,88.36,200.00,206.0,239.99,160.50,418.00,428.72,202.00,163.20,...,66.75,388.00,305.1,616.10,726.48,942.60,917.25,543.06,8.44,2
10147994,38.11,39.52,218.40,227.9,224.70,229.69,226.00,504.29,4.24,256.20,...,1289.95,861.59,741.1,104.62,-52.00,-230.21,-374.16,158.00,-11.00,0
10152808,420.00,1030.00,510.00,423.3,877.50,1157.44,709.00,995.00,1015.00,515.00,...,800.69,571.52,775.0,957.00,1266.00,910.80,1132.00,674.48,818.62,0


In [102]:
X_test = np.array(processed_test)

In [103]:
X_test.shape

(5100, 50)

In [109]:
predict_lgb = final_lgb.predict(X_test)
predict_lgb = np.array([0 if i < 0.6 else 1 for i in predict_lgb])

In [110]:
processed_test['Default'] = predict_lgb

In [111]:
results = processed_test.reset_index()[['ID_CPTE', 'Default']]

In [112]:
submission = pd.read_csv('../raw_data/performance_test.csv')

In [113]:
submission.head()

,ID_CPTE,PERIODID_MY,Default
0,71424379,2014-12-01,NaN
1,64887111,2015-12-01,NaN
2,69431075,2014-12-01,NaN
3,31823308,2016-12-01,NaN
4,39407834,2012-12-01,NaN


In [114]:
results.head()

,ID_CPTE,Default
0,10074849,0
1,10086539,0
2,10140908,1
3,10147994,1
4,10152808,0


In [115]:
submission = submission[['ID_CPTE', 'Default']].merge(results, on='ID_CPTE')

In [116]:
submission = submission[['ID_CPTE', 'Default_y']]

In [117]:
submission = submission.rename(columns={'Default_y': 'Default'})

In [118]:
submission.head()

,ID_CPTE,Default
0,71424379,0
1,64887111,0
2,69431075,0
3,31823308,0
4,39407834,0


In [119]:
submission.to_csv('submission.csv', index_label=False)

In [59]:
tmp = pd.read_csv('submission.csv')

In [168]:
len(results)

5100

In [169]:
performance_test.head()

,ID_CPTE,PERIODID_MY,Default
0,71424379,2014-12-01,NaN
1,64887111,2015-12-01,NaN
2,69431075,2014-12-01,NaN
3,31823308,2016-12-01,NaN
4,39407834,2012-12-01,NaN
